In [1]:
# Importation des librairies
import json
import os
import numpy as np
import pandas as pd
from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import torch
from sklearn.preprocessing import StandardScaler

In [2]:
pd.set_option('future.no_silent_downcasting', True)

In [17]:
def read_csv_files_to_dict(base_path, dimension, file_prefix, missing_percentages, strategies=None):
    dataframes_dict = {}

    for percentage in missing_percentages:
        percentage_key = f"{int(percentage * 100)}%"

        if dimension == 'Completeness':
            dataframes_dict[percentage_key] = {}

            if strategies is not None:
                for strategy_name, strategy_func, subfolder in strategies:
                    file_path = f"{base_path}/{dimension}/{subfolder}/{file_prefix}_{percentage_key}.csv"

                    try:
                        df = pd.read_csv(file_path)
                        dataframes_dict[percentage_key][strategy_name] = df
                    except FileNotFoundError:
                        print(f"File not found: {file_path}")
                        dataframes_dict[percentage_key][strategy_name] = None
            else:
                print("No strategies provided for Completeness dimension.")
        elif dimension == 'Unicity':
            file_path = f"{base_path}/{dimension}/{file_prefix}_{percentage_key}_2x.csv"
            try:
                df = pd.read_csv(file_path)
                dataframes_dict[percentage_key] = df
            except FileNotFoundError:
                print(f"File not found: {file_path}")
                dataframes_dict[percentage_key] = None
        else:
            file_path = f"{base_path}/{dimension}/{file_prefix}_{percentage_key}.csv"
            try:
                df = pd.read_csv(file_path)
                dataframes_dict[percentage_key] = df
            except FileNotFoundError:
                print(f"File not found: {file_path}")
                dataframes_dict[percentage_key] = None

    return dataframes_dict

def update_json_results(output_path, model_name, pollution_percentage, results):
    # Charger le fichier JSON existant, ou initialiser une nouvelle structure si le fichier n'existe pas
    if os.path.exists(output_path):
        with open(output_path, 'r') as json_file:
            results_dict = json.load(json_file)
    else:
        results_dict = {
            "models": []
        }

    # Trouver ou ajouter l'entrée pour le modèle spécifié
    model_entry = next((model for model in results_dict["models"] if model["model"] == model_name), None)

    if not model_entry:
        model_entry = {
            "model": model_name,
            "pollution_metrics": []
        }
        results_dict["models"].append(model_entry)

    # Chercher si le pourcentage de pollution existe déjà pour ce modèle
    existing_entry = next((item for item in model_entry["pollution_metrics"] if item["pollution_percentage"] == pollution_percentage), None)

    if existing_entry:
        # Si le pourcentage de pollution existe, remplacer les métriques
        existing_entry["metrics"] = results
    else:
        # Sinon, ajouter une nouvelle entrée pour ce pourcentage
        model_entry["pollution_metrics"].append({
            "pollution_percentage": pollution_percentage,
            "metrics": results
        })

    # Écrire les résultats mis à jour dans le fichier JSON
    with open(output_path, 'w') as json_file:
        json.dump(results_dict, json_file, indent=4)

    print(f"Results saved to {output_path}")

def update_json_results_completeness(output_path, model_name, strategy, pollution_percentage, results):
    # Charger le fichier JSON existant, ou initialiser une nouvelle structure si le fichier n'existe pas
    if os.path.exists(output_path):
        with open(output_path, 'r') as json_file:
            results_dict = json.load(json_file)
    else:
        results_dict = {
            "models": [
                {
                    "model": model_name,
                    "imputation_strategies": []
                }
            ]
        }

    # Trouver ou ajouter l'entrée pour le modèle spécifié
    model_entry = next((model for model in results_dict["models"] if model["model"] == model_name), None)

    if not model_entry:
        model_entry = {
            "model": model_name,
            "imputation_strategies": []
        }
        results_dict["models"].append(model_entry)

    # Trouver ou créer l'entrée pour la stratégie d'imputation actuelle
    strategy_entry = next((item for item in model_entry["imputation_strategies"] if item["strategy"] == strategy), None)

    if not strategy_entry:
        # Ajouter une nouvelle stratégie d'imputation si elle n'existe pas encore
        strategy_entry = {
            "strategy": strategy,
            "pollution_metrics": []
        }
        model_entry["imputation_strategies"].append(strategy_entry)

    # Vérifier si le pourcentage de pollution existe déjà pour cette stratégie
    existing_entry = next((item for item in strategy_entry["pollution_metrics"] if item["pollution_percentage"] == pollution_percentage), None)

    if existing_entry:
        # Si le pourcentage de pollution existe, remplacer les métriques
        existing_entry["metrics"] = results
    else:
        # Sinon, ajouter une nouvelle entrée pour ce pourcentage
        strategy_entry["pollution_metrics"].append({
            "pollution_percentage": pollution_percentage,
            "metrics": results
        })

    # Écrire les résultats mis à jour dans le fichier JSON
    with open(output_path, 'w') as json_file:
        json.dump(results_dict, json_file, indent=4)

    print(f"Results saved to {output_path}")


def prepare_fps_data(fps_df):
    # Conversion des colonnes catégorielles en chaîne de caractères
    fps_df['GpuOpenCL'] = fps_df['GpuOpenCL'].astype(str)
    fps_df['GpuShaderModel'] = fps_df['GpuShaderModel'].astype(str)

    # Encodage des variables catégorielles
    fps_df = pd.get_dummies(fps_df, drop_first=True)

    # Séparer les variables indépendantes (X) de la variable cible (y)
    X = fps_df.drop('FPS', axis=1)
    y = fps_df['FPS']

    # Identification des colonnes à normaliser (exclure les colonnes booléennes et dummies)
    numeric_cols = X.select_dtypes(include=[np.number]).columns
    cols_to_scale = [col for col in numeric_cols if len(X[col].unique()) > 2]

    # Normalisation des colonnes sélectionnées
    scaler = StandardScaler()
    X[cols_to_scale] = scaler.fit_transform(X[cols_to_scale])

    # Conversion en float32
    X = X.astype(np.float32)
    y = y.astype(np.float32)

    return X, y

def train_and_evaluate(X, y):
    device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
    model = TabNetRegressor(
        device_name=device.type,
        n_d=32,  # Augmentation de la taille des couches
        n_a=32,  # Augmentation de la taille des couches
        optimizer_fn=torch.optim.Adam,
        optimizer_params=dict(lr=1e-4),  # Réduction du taux d'apprentissage
        scheduler_params={"step_size":10, "gamma":0.8},
        scheduler_fn=torch.optim.lr_scheduler.StepLR,
    )

    kfold = KFold(n_splits=5, shuffle=True, random_state=42)

    mse_scores = []
    mae_scores = []
    r2_scores = []

    for train_index, val_index in kfold.split(X):
        X_train, X_val = X.iloc[train_index].values, X.iloc[val_index].values
        y_train, y_val = (
            y.iloc[train_index].values.reshape(-1, 1).astype(np.float32),
            y.iloc[val_index].values.reshape(-1, 1).astype(np.float32)
        )

        model.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            eval_name=['val'],
            eval_metric=['mae'],
            max_epochs=300,
            patience=30,
            batch_size=2048,
            virtual_batch_size=512,
            num_workers=1,
            drop_last=False
        )

        y_pred = model.predict(X_val)

        mse_scores.append(mean_squared_error(y_val, y_pred))
        mae_scores.append(mean_absolute_error(y_val, y_pred))
        r2_scores.append(r2_score(y_val, y_pred))

    results = {
        "mean_squared_error": np.mean(mse_scores),
        "mean_absolute_error": np.mean(mae_scores),
        "r2_score": np.mean(r2_scores)
    }

    return results

In [18]:
df = pd.read_csv('../../Data/Regression/fps in video games/fps-in-video-games_clean.csv')

X, y = prepare_fps_data(df)

results = train_and_evaluate(X, y)

/var/folders/qc/m20c7_k95j52q7xcg6lwyd7w0000gn/T/ipykernel_9036/253039370.py:1: DtypeWarning: Columns (25,30) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../../Data/Regression/fps in video games/fps-in-video-games_clean.csv')
/opt/anaconda3/envs/Impact-of-data-quality-on-machine-learning-models/lib/python3.11/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : mps
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 26828.2645| val_mae: 138.35609436035156|  0:00:23s
epoch 1  | loss: 26767.60703| val_mae: 138.14724731445312|  0:00:46s
epoch 2  | loss: 26712.66267| val_mae: 137.9803924560547|  0:01:09s
epoch 3  | loss: 26631.26387| val_mae: 137.70384216308594|  0:01:32s
epoch 4  | loss: 26570.68639| val_mae: 137.54266357421875|  0:01:55s
epoch 5  | loss: 26511.83873| val_mae: 137.28414916992188|  0:02:17s
epoch 6  | loss: 26438.65023| val_mae: 136.86758422851562|  0:02:40s
epoch 7  | loss: 26365.09318| val_mae: 136.60360717773438|  0:03:02s
epoch 8  | loss: 26287.53536| val_mae: 136.59112548828125|  0:03:25s
epoch 9  | loss: 26218.32362| val_mae: 136.33079528808594|  0:03:47s
epoch 10 | loss: 26159.32405| val_mae: 135.96156311035156|  0:04:10s
epoch 11 | loss: 26087.70128| val_mae: 135.4628448486328|  0:04:32s
epoch 12 | loss: 26020.0011| val_mae: 135.3905792236328|  0:04:55s
epoch 13 | loss: 25933.48998| val_mae: 134.8812255859375|  0:05:17s
epoch 14 | loss: 25866.60702| val_mae: 1

KeyboardInterrupt: 

## Completeness

In [ ]:
# Utilisation de la fonction dans votre code existant
missing_percentages = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]

# Strategy :
strategy_delete = {
    'numerical': 'delete',
    'categorical': 'delete'
}

strategy_mean_mode = {
    'numerical': 'mean',
    'categorical': 'mode'
}

strategy_median_new = {
    'numerical': 'median',
    'categorical': 'new'
}

strategy_decision_tree = {
    'numerical': 'decision_tree',
    'categorical': 'decision_tree'
}

strategy_mean_new = {
    'numerical': 'mean',
    'categorical': 'new'
}

strategy_knn_mode = {
    'numerical': 'knn',
    'categorical': 'mode'
}

strategies = [
    ("Mean and Mode", strategy_mean_mode, "Mean and Mode"),
    ("Median and New", strategy_median_new, "Median and New"),
    ("Decision Tree", strategy_decision_tree, "Decision Tree"),
    ("Mean and New", strategy_mean_new, "Mean and New")
]

fps_df_dict = read_csv_files_to_dict('../../Data/Regression/fps in video games', 'Completeness', 'fps', missing_percentages, strategies)

# Chemin du fichier JSON où les résultats seront enregistrés
output_path = "../../Results/FPS in video games/Completeness.json"

# Boucle sur les DataFrames pour chaque pourcentage de pollution
for key, fps_df_strategies in fps_df_dict.items():
    for strategy_name, fps_df in fps_df_strategies.items():
        X, y = prepare_fps_data(fps_df)

        results = train_and_evaluate(X, y)

        # Extraire le pourcentage de pollution à partir de la clé
        pollution_percentage = float(key.replace('%', ''))

        # Mise à jour des résultats dans le fichier JSON en utilisant la fonction
        update_json_results_completeness(output_path, "TabNET", strategy_name, pollution_percentage, results)

## Consistent Representation

In [ ]:
# Utilisation de la fonction dans votre code existant
pollution_percentage_levels = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
fps_df_dict = read_csv_files_to_dict('../../Data/Regression/fps in video games', 'Consistent Representation', 'fps', pollution_percentage_levels)

# Chemin du fichier JSON où les résultats seront enregistrés
output_path = "../../Results/FPS in video games/Consistent Representation.json"

# Boucle sur les DataFrames pour chaque pourcentage de pollution
for key, fps_df in fps_df_dict.items():
    X, y = prepare_fps_data(fps_df)

    results = train_and_evaluate(X, y)

    # Extraire le pourcentage de pollution à partir de la clé
    pollution_percentage = float(key.replace('%', ''))

    # Mise à jour des résultats dans le fichier JSON
    update_json_results(output_path, "CatBoost", pollution_percentage, results)

## Feature Accuracy

In [ ]:
pollution_percentage_levels = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
fps_df_dict = read_csv_files_to_dict('../../Data/Regression/fps in video games', 'Feature Accuracy', 'fps', pollution_percentage_levels)

# Chemin du fichier JSON où les résultats seront enregistrés
output_path = "../../Results/FPS in video games/Feature Accuracy.json"

# Boucle sur les DataFrames pour chaque pourcentage de pollution
for key, fps_df in fps_df_dict.items():
    X, y = prepare_fps_data(fps_df)

    results = train_and_evaluate(X, y)

    # Extraire le pourcentage de pollution à partir de la clé
    pollution_percentage = float(key.replace('%', ''))

    # Mise à jour des résultats dans le fichier JSON
    update_json_results(output_path, "CatBoost", pollution_percentage, results)

## Target Accuracy

In [ ]:
pollution_percentage_levels = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
fps_df_dict = read_csv_files_to_dict('../../Data/Regression/fps in video games', 'Target Accuracy', 'fps', pollution_percentage_levels)

# Chemin du fichier JSON où les résultats seront enregistrés
output_path = "../../Results/FPS in video games/Target Accuracy.json"

# Boucle sur les DataFrames pour chaque pourcentage de pollution
for key, fps_df in fps_df_dict.items():
    X, y = prepare_fps_data(fps_df)

    results = train_and_evaluate(X, y)

    # Extraire le pourcentage de pollution à partir de la clé
    pollution_percentage = float(key.replace('%', ''))

    # Mise à jour des résultats dans le fichier JSON
    update_json_results(output_path, "CatBoost", pollution_percentage, results)

## Unicity

In [ ]:
pollution_percentage_levels = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
fps_df_dict = read_csv_files_to_dict('../../Data/Regression/fps in video games', 'Unicity', 'fps', pollution_percentage_levels)

# Chemin du fichier JSON où les résultats seront enregistrés
output_path = "../../Results/FPS in video games/Unicity.json"

# Boucle sur les DataFrames pour chaque pourcentage de pollution
for key, fps_df in fps_df_dict.items():
    X, y = prepare_fps_data(fps_df)

    results = train_and_evaluate(X, y)

    # Extraire le pourcentage de pollution à partir de la clé
    pollution_percentage = float(key.replace('%', ''))

    # Mise à jour des résultats dans le fichier JSON
    update_json_results(output_path, "CatBoost", pollution_percentage, results)